In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [ ]:
df_pop= pd.read_csv('../../../data/popular_data_with_features3.csv', index_col=0)
df_pop.head()

In [ ]:
# get all keys
ilocs = df_pop.groupby(["infobox_key","property_name"],sort=False).count()["timestamp"]
ilocs=ilocs[:1000]

In [ ]:
ilocs

In [ ]:
import time

def generate_predictions(ilocs):
    predictions=[]
    start=0
    for iloc in tqdm(ilocs):
        tmp = time.time()
        # sample = df_pop[(df_pop['infobox_key']==key[0]) & (df_pop['property_name']==key[1])]
        sample = df_pop.iloc[start:start+iloc]
        start+=iloc
        # print(1,time.time()-tmp)
        tmp = time.time()
        sample.drop(columns=["infobox_key","page_title","property_name","template"], inplace=True)
        # print(2,time.time()-tmp)
        tmp = time.time()
        X = sample.iloc[:-1].drop(columns=['days_until_next_change','timestamp'])
        # print(3,time.time()-tmp)
        tmp = time.time()
        y = sample.iloc[:-1]['days_until_next_change']
        # print(4,time.time()-tmp)
        tmp = time.time()
        x_last = sample.iloc[-1].drop(['days_until_next_change','timestamp']).values.reshape(1,-1)
        # print(5,time.time()-tmp)
        tmp = time.time()
        x_last_timestamp = datetime.datetime.strptime(sample.iloc[-1]["timestamp"],"%Y-%m-%d").date()
        # print(6,time.time()-tmp)
        tmp = time.time()
        clf = RandomForestClassifier(random_state=0, n_estimators=10)
        # print(7,time.time()-tmp)
        tmp = time.time()
        clf.fit(X, y)
        # print("fit",time.time()-tmp)
        tmp = time.time()
        predictions.append(clf.predict(x_last)[0])
        print("pred",time.time()-tmp)

    return predictions

In [ ]:
import warnings
warnings.filterwarnings('ignore')

preds=generate_predictions(ilocs)

In [ ]:
abc=ilocs

In [ ]:
preds_df=pd.DataFrame(abc.index.to_list(),columns=["infobox_key","property_name"])
preds_df["pred"]=preds
preds_df

In [ ]:
300000/15/360/100